In [1]:
import pyautogui
import numpy as np
import time
import pydirectinput
import cv2
import mss
import time
import pywinauto
import win32gui
import torch
import torch.nn as nn
import torchvision
from KeyPress import PressKey, ReleaseKey
from collections import deque

# Cargamos el modelo

In [3]:
keys_dict = {"up": "x", "down": "b", "left": "y", "right": "a", "none": ""}
keys_dict_di = {"up": 0x2D, "down": 0x30, "left": 0x15, "right": 0x1E, "none": 0x2D}

In [4]:
classes = ['down', 'none', 'right']
classes

['down', 'none', 'right']

In [5]:
#Primero hay que definir el modelo
class TemporalModel(nn.Module):
    def __init__(self):
        super(TemporalModel, self).__init__()
        self.convolutions = nn.Sequential(*[
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding = 1, stride = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding = 1, stride = 1),
        nn.ReLU(),
        nn.MaxPool2d(4),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding = 1, stride = 1),
        nn.ReLU(),
        nn.MaxPool2d(4),
        nn.Flatten(),
        nn.Linear(64*14*18, 100),
        nn.ReLU(),
    ])
        self.hidden_size = 50
        self.lstm = nn.LSTM(input_size = 100, hidden_size = self.hidden_size, num_layers = 1, batch_first = True, dropout=0.5)
        #Definimos el estado oculto
        self.hidden_cell = (torch.zeros(1,1,self.hidden_size).to(device),
                            torch.zeros(1,1,self.hidden_size).to(device))
        
        self.final_linear = nn.Linear(self.hidden_size, 3)

    def forward(self, x):      
        pred = torch.empty(x.shape[0],3,100).to(device)

        for i in range(x.shape[1]):
            pred[:,i,:] = self.convolutions(x[:,i,:,:,:])

        pred, self.hidden_cell = self.lstm(pred, self.hidden_cell)


        #Tenemos que hacerle detach al hidden_cell porque eso no se ajusta
        self.hidden_cell = tuple(a.detach() for a in self.hidden_cell)

        # Nos quedamos solamente con el elemento que corresponde al último
        # instante temporal
        pred = self.final_linear(pred[:,-1,:])

        return pred

In [6]:
device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
model_custom = TemporalModel()
#Cargamos los pesos del modelo entrenado
model_custom.load_state_dict(torch.load("ModeloLSTM_11_12_13_522_BotonRojo.pth"))
model_custom.eval()
model_custom.to(device)

C:\Users\vila_\anaconda3\envs\Master\lib\site-packages\torch\nn\modules\rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


TemporalModel(
  (convolutions): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=16128, out_features=100, bias=True)
    (10): ReLU()
  )
  (lstm): LSTM(100, 50, batch_first=True, dropout=0.5)
  (final_linear): Linear(in_features=50, out_features=3, bias=True)
)

In [8]:
def transform_image(img):
    """
    Transformaciones necesarias para poder meter las imágenes en la red

    Argumentos:
    img -> Imagen

    Returns:
    img -> Imagen transformada
    """
    #La pasamos a RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    #Recortamos solo la sección que nos interesa
    img = img[49:-12,20:-20]
    #Ajustamos la imagen al tamaño que espera la red
    img = cv2.resize(img, dsize = (293,224))
    #Transformamos la imagen a Tensor
    img = torchvision.transforms.ToTensor()(img)
    return img

In [9]:
def deque_to_tensor(deque):
    """
    Cogemos el buffer de imágenes (deque) y lo convertimos en un tensor con la forma adecuada.

    Argumentos:
    deque -> deque de imágenes (buffer)

    Returns:
    tensor_ready -> torch.Tensor listo para entrar a la red LSTM
    """
    tensor_ready = torch.empty(1,3, 3, 224, 293)
    for i in range(len(img_buffer)):
        tensor_ready[:,i,:] = img_buffer[i]
    return tensor_ready

In [10]:
def predict_action(model, data, classes):
    """
    Predecimos la clase correspondiente para una imagen dada con el modelo deseado.
    El preprocesado de la imagen se hace dentro de la función, no fuera.

    Args:
    model -> Modelo que queremos usar para hacer la predicción
    data -> Conjunto de imágenes que usamos para predecir
    classes -> Lista de las posibles clases

    Returns:
    Acción en formato str
    """

    data = data.to(device)
    with torch.no_grad():
        pred = model(data)
    return classes[torch.softmax(pred, dim = 1).argmax(dim = 1)]

# Interacción con la pantalla

In [12]:
#Left, Top, Right, Bottom
window_coords = win32gui.GetWindowRect(pywinauto.findwindows.find_window(title_re = "Super Mario"))
window_coords

(956, 269, 1499, 776)

In [13]:
left, top, right, bottom = window_coords

In [14]:
width = right - left
height = bottom - top

In [15]:
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
  
# org 
org = (50, 50) 
  
# fontScale 
fontScale = 1
   
# Blue color in BGR 
color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 2

In [17]:
keys_dict_di = {"up": 0x2D, "down": 0x30, "left": 0x15, "right": 0x1E, "none": 0x2D}

In [19]:
time.sleep(2)

img_buffer = deque(maxlen=3)

with mss.mss() as  sct:
    monitor = {"top": top, "left": left, "width": width, "height": height}

    while "Screen Recording": #Aquí da igual lo que pongas porque una string es como si fuera un True
        last_time = time.time()

        #Esto coge las imágenes en BGRA
        sct_img = sct.grab(monitor)
        img = np.array(sct_img)
        img_transformed = transform_image(img)
        img_buffer.append(img_transformed)
        
        if len(img_buffer) < 3:
            continue
        #Transformamos el deque a un tensor
        tensor_ready = deque_to_tensor(img_buffer)

        #Predecimos la acción con nuestro modelo
        action = predict_action(model_custom, tensor_ready, classes)

        # pydirectinput.press(keys_dict[action], _pause = False)

        text = "Action: {} fps: {}".format(action, 1 / (time.time() - last_time))
        tries = 0
        if action != "none":
            tries = 4
            for i in range(tries):
                PressKey(keys_dict_di[action])
                time.sleep(0.001)
                ReleaseKey(keys_dict_di[action])

        img = cv2.putText(img, text, org, font,  
                   fontScale, color, thickness, cv2.LINE_AA) 

        cv2.imshow("Cuac", img)

        #Press "q" to quit
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            break